#### In this Jupyter Notebook we will display the results after training of the model

##### Importing the libraries

In [ ]:
import os
from os.path import exists
from glob import glob
import torch
from monai.networks.nets import UNet
from monai.networks.layers import Norm
from monai.losses import DiceLoss
from monai.data import Dataset, CacheDataset,DataLoader
from monai.utils import set_determinism
from monai.transforms import (
    Compose,
    LoadImaged,
    EnsureChannelFirstd,
    ScaleIntensityRanged,
    RandAffined,
    RandRotated,
    RandGaussianNoised,
    CropForegroundd,
    Orientationd,
    Resized,
    ToTensord,
    Spacingd,
)

##### Setting the path to the working directories

In [ ]:
# The input paths for the prepared nifti files
nif_path = ['data_set_group_nif/nif_files_testing/images',
            'data_set_group_nif/nif_files_testing/labels',
            'data_set_group_nif/nif_files_training/images',
            'data_set_group_nif/nif_files_training/labels',]

##### Define the function for

##### Process the data

In [ ]:
data_in = preprocess_data(nif_path, batch_size=1, spatial_size=(512, 512, 16))

##### Setting the device for training

In [ ]:
# We do the training on the GPU
device = torch.device("cuda:0")
print(device)

##### Initialize the model

In [ ]:
model = UNet(
    spatial_dims=3,
    in_channels=1,
    out_channels=2,
    channels=(16, 32, 64, 128, 256),
    strides=(2, 2, 2, 2),
    num_res_units=2,
    norm=Norm.BATCH,
).to(device)

##### Initialize the loss function and the optimizer

In [ ]:
loss_function = DiceLoss(to_onehot_y=True, sigmoid=True, squared_pred=True)
optimizer = torch.optim.Adam(model.parameters(), 1e-5, weight_decay=1e-5, amsgrad=True)

##### Train the model

In [ ]:
os.makedirs('post_training', exist_ok=true)

train(model,
      data_in,
      loss_function,
      optimizer,
      10,
      '../post_training_best',
      test_interval=100,
      device=device
)
